In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers,optimizers
import random
from datetime import datetime
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
Data=pd.read_csv('../input/data-science-and-stem-salaries/Levels_Fyi_Salary_Data.csv')
Data.head()
Len=len(Data)

FileNotFoundError: ignored

In [ ]:
Data.info()

In [ ]:
Data.describe()

In [ ]:
Cor_Mat=Data.corr()
plt.figure(figsize=(15,15))
sns.heatmap(Cor_Mat, annot=True)

In [ ]:
Data.isnull().sum()

In [ ]:
Data.shape

In [ ]:
Data1 = Data.drop(['level','yearsatcompany','tag','basesalary','stockgrantvalue','bonus','gender','otherdetails',], axis=1)
Data1.head(5)

In [ ]:
Data2 = Data1.drop(Data1.iloc[:,6:], axis=1)
Data2.head(5)

In [ ]:
Data2.info()

In [ ]:
Data2.rename(columns={'timestamp':'year','title':'position','totalyearlycompensation':'salary','yearsofexperience':'experience'},inplace=True)
Data2

In [ ]:
Data2[Data2.duplicated()].shape

In [ ]:
Data3=Data2.drop_duplicates()
Data3.head()

In [ ]:
Data3.shape

In [ ]:
Data4=Data3.copy(deep=True)
Data4[Data4['company'].isnull()]

In [ ]:
Data5=Data4.dropna()
Data5.head()

In [ ]:
Data5.isnull().sum()

In [ ]:
for i in range(Len):
    try:
        X=Data5['year'][i].split(' ')
        Data5['year'][i]=int(X[0][-4:])
    except:
        pass

In [ ]:
Data6=Data5.copy(deep=True)
Data6

In [ ]:
#pd.options.display.max_columns
pd.reset_option("all")
Data6.info()

In [ ]:
Data6['year']=Data6['year'].astype(int)
Data6.info()

In [ ]:
#To check outliers if any
plt.figure(figsize=(5,5))
sns.boxplot(x=Data6['salary'])

In [ ]:
Data6['year'].unique()

In [ ]:
len(Data6['company'].unique())

In [ ]:
len(Data6['position'].unique())

In [ ]:
len(Data6['location'].unique())

In [ ]:
Data7=Data6.copy(deep=True)
le_company= preprocessing.LabelEncoder()
le_company.fit(Data7['company'])
list(le_company.classes_)
Data7['company']=le_company.transform(Data7['company'])
list(le_company.inverse_transform([100]))
Data7.head()

In [ ]:
le_position= preprocessing.LabelEncoder()
le_position.fit(Data7['position'])
list(le_position.classes_)
Data7['position']=le_position.transform(Data7['position'])
#list(le_position.inverse_transform([0]))
Data7.head()

In [ ]:
le_loc= preprocessing.LabelEncoder()
le_loc.fit(Data7['location'])
list(le_loc.classes_)
Data7['location']=le_loc.transform(Data7['location'])
#list(le_loc.inverse_transform([0]))
Data7.head()

In [ ]:
Data7.describe()

In [ ]:
Cor_Mat1=Data7.corr()
plt.figure(figsize=(15,15))
sns.heatmap(Cor_Mat1,annot=True)

In [ ]:
X=Data7.copy(deep=True)
X.drop(columns=['salary'],axis=1,inplace=True)
X.head()

In [ ]:
y=Data7['salary']
y.head()

In [ ]:
for i in range(len(X.iloc[0,:])):
    plt.figure(figsize=(5,5))
    plt.title(X.columns[i])
    plt.scatter(X.iloc[:,i],y)
print(i)

In [ ]:
X_train=X.sample(frac=0.8,random_state=42)
X_test=X.drop(X_train.index)
y_train=y.sample(frac=0.8,random_state=42)
y_test=y.drop(y_train.index)

In [ ]:
# min max scaling the variables
# scaler =  MinMaxScaler()
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))
L=len(X_train.iloc[0])
print(L)
X_train.head()

In [ ]:
model = Sequential()
model.add(Dense(L, input_dim=L, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=500)

In [ ]:
y_test_pred= model.predict(X_test).flatten()
for i in range(10):
    print(y_test.iloc[i],'-',int(y_test_pred[i]),'Diff=',abs(y_test.iloc[i]-int(y_test_pred[i])),'%diff=',int(abs(y_test.iloc[i]-int(y_test_pred[i]))*100/y_test.iloc[i]))

In [ ]:
Data6.head(1)
Data6['company'].value_counts()

# Q 1: How much you would get if I join for a position based on number of experiences and location?

In [ ]:
Position='Product Manager'
Experience=1.5
Location='Redwood City, CA'
def Predict_Salary1(Pos,Exp,Loc):
    Y=2022
    C=int(le_company.transform(['Amazon']))
    P=int(le_position.transform([Pos]))
    L=int(le_loc.transform([Loc]))
    E=Exp
    data ={'year':[Y],'company':[C],'position':[P],'location':[L],'experience':[E]}
    df = pd.DataFrame(data)
    print('Given Data:','\nPosition=',Pos,'\nExperience=',Exp,'\nLocation=',Loc)
    y_test_pred= model.predict(df).flatten()
    print('-----------------------------------')
    print('Expected Salary=',int(y_test_pred))
    pass

Predict_Salary1(Position,Experience,Location)

# Q 2: How much you would get if you accept an offer for a position from X company based on number of experiences and location?

In [ ]:
Company='Oracle'
Position='Product Manager'
Experience=1.5
Location='Redwood City, CA'
def Predict_Salary2(Com,Pos,Exp,Loc):
    Y=2022
    C=int(le_company.transform([Com]))
    P=int(le_position.transform([Pos]))
    L=int(le_loc.transform([Loc]))
    E=Exp
    data ={'year':[Y],'company':[C],'position':[P],'location':[L],'experience':[E]}
    df = pd.DataFrame(data)
    print('Given Data:','\nCompany=',Com,'\nPosition=',Pos,'\nExperience=',Exp,'\nLocation=',Loc)
    y_test_pred= model.predict(df).flatten()
    print('-----------------------------------')
    print('Expected Salary=',int(y_test_pred))
    pass

Predict_Salary2(Company,Position,Experience,Location)

# Q 3: How much you will be getting for a position after Y years joining to X company?

In [ ]:
Company='Amazon'
Position='Product Manager'
After_Years=5
def Predict_Salary3(Com,Pos,Aft):
    Y=2022+Aft
    C=int(le_company.transform([Com]))
    P=int(le_position.transform([Pos]))
    L=int(le_loc.transform(['Seattle, WA']))
    E=0
    data ={'year':[Y],'company':[C],'position':[P],'location':[L],'experience':[E]}
    df = pd.DataFrame(data)
    print('Given Data:','\nCompany=',Com,'\nPosition=',Pos,'\nAfter Years=',Aft)
    y_test_pred= model.predict(df).flatten()
    print('-----------------------------------')
    print('Expected Salary=',int(y_test_pred))
    pass

Predict_Salary3(Company,Position,After_Years)